Hi，今天我们来一个新的project，这次用的数据集是一个猴痘病的图像识别任务。

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision
from torchvision import transforms, datasets

import os,PIL,pathlib

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cpu')

#### pathlib.Path
这里有一个我们不太常用的函数是**pathlib.Path(path)**，我们可以将我们的path给到pathlib.Path中，然后就可以用pathlib.Path函数的一些操作快捷操作path及path的子路径。

像 **pathlib.Path(data_dir).glob("\*")** 这个是非递归匹配data_dir下的所有文件及文件夹，而像 **pathlib.Path(data_dir).rglob("\*")** 这个是递归式的匹配data_dir下所有的文件及文件夹。

In [2]:
import os,PIL,random,pathlib

data_dir = './4-data/'
data_dir = pathlib.Path(data_dir)

data_paths = list(data_dir.glob('*'))
#这里用list后会得到[WindowsPath('4-data/Monkeypox'), WindowsPath('4-data/Others')]
classNames = [str(path).split("\\")[1] for path in data_paths]
#这里的for path in data_paths会提取每一个list[i]，
#这里用了str(path).split("\\")[1]，会得到['Monkeypox', 'Others']
classNames

['Monkeypox', 'Others']

#### note
这里可能很多人都会觉得，不知道该怎么一步步把classNames提取出来，其实我觉得99.99999%的人一开始都不知道，一开始我们能走到的都是data_paths那一层，也是从这一层慢慢往下走才知道，噢！还要再str等往下拆几步。初学者切忌再数据处理上纠结太多。

#### transforms.Compose
当我们拿到一些图片数据的时候，这些数据往往是什么形状都有且尚未处于tenosr类型，因此我们需要用transforms.Compose来对图像做预处理。

1、**transforms.Resize([224, 224])** ：是将所有的图像大小改为244*244个像素点；

2、**transforms.ToTensor()** ：是将PIL Image或numpy.ndarray转换为tensor，并且会将像素值归一化到(0,1)；

3、**transforms.Normalize()** ：给定一个mean&std值，求tensor数据的标准正态分布，使模型更容易收敛。

#### datasets.ImageFolder
因为今天我们做的是分类任务，因此我们的目录结构肯定是root/A_folder/xxx.jpg以及root/B_folder/xxx.jpg，如果是二分类就到此为止，如果是三分类就继续往下有C_folder。因此我们datasets.ImageFolder的核心作用是：

1、 **加载数据集** ： 每个文件夹代表一个类别，它将使用文件夹的名称作为类别的标签，并加载文件夹中的所有图像。

2、 **图像转换** ：我们可以在ImageFolder中调用我们创建好的transforms.Compose()，以此来构建处理好的数据集。


In [3]:
total_datadir = './4-data/'

# 关于transforms.Compose的更多介绍可以参考：https://blog.csdn.net/qq_38251616/article/details/124878863
train_transforms = transforms.Compose([
    transforms.Resize([224, 224]),  # 将输入图片resize成统一尺寸
    transforms.ToTensor(),          # 将PIL Image或numpy.ndarray转换为tensor，并归一化到[0,1]之间
    transforms.Normalize(           # 标准化处理-->转换为标准正太分布（高斯分布），使模型更容易收敛
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225])  # 其中 mean=[0.485,0.456,0.406]与std=[0.229,0.224,0.225] 从数据集中随机抽样计算得到的。
])

total_data = datasets.ImageFolder(root=total_datadir,transform=train_transforms)
total_data

Dataset ImageFolder
    Number of datapoints: 2142
    Root location: ./4-data/
    StandardTransform
Transform: Compose(
               Resize(size=[224, 224], interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [4]:
total_data.class_to_idx

{'Monkeypox': 0, 'Others': 1}

#### torch.utils.data.random_split
当我们处理好上面的数据集之后，我们需要分割数据，比如将数据分割成训练集（70%）、验证集（15%）、测试集（15%）或者训练集（80%）和验证集（20%）。我们使用的函数就是 **torch.utils.data.random_split** 。如果现在要拆分成训练集、验证集以及测试集的话只需要在等号右边放三个变量以及在random_split函数分割数据大小里面由两个数据量改成三个数据量。

In [4]:
train_size = int(0.8 * len(total_data))
test_size  = len(total_data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(total_data, [train_size, test_size])
train_dataset, test_dataset

(<torch.utils.data.dataset.Subset at 0x2409ca2d2e0>,
 <torch.utils.data.dataset.Subset at 0x2409ca2dca0>)

In [6]:
train_size,test_size

(1713, 429)

#### torch.utils.data.DataLoader
我们刚才通过 **train_transforms** 做了每一个数据的预处理，用了 **torch.utils.data.random_split** 分割了数据集。现在我们要用 **torch.utils.data.DataLoader** 去对数据进行批量处理。

**torch.utils.data.DataLoader** 这个函数比较常用的参数就是dataset、batch_size、shuffle、num_workers以及drop_last，这里没有写drop_last，default false。

当我们用torch.utils.data.DataLoader处理完数据后，其实是将input图像处理成了[batch_size,channel,height,width]。

#### note
对于tensor、numpy数据，都可以用.shape/.dtype去看数据维度以及数据类型。

In [5]:
batch_size = 32

train_dl = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=1)
test_dl = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=1)
for X, y in test_dl:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([32, 3, 224, 224])
Shape of y:  torch.Size([32]) torch.int64


#### nn.Conv2d
这是一个卷积函数，用于处理图像数据，从图像中提取特征。其常用参数有：

1、 **in_channels** ：输入图像的通道数，例如，灰度图像为 1，RGB 图像为 3。

2、 **out_channels** ：输出特征图的通道数，也是过滤器的数量。

3、 **kernel_size** ：卷积核的大小。可以是整数或二元组来分别指定高度和宽度。

4、 **stride** ：卷积核的步长，即每次卷积核滑动的像素数，默认为 1。

5、 **padding** ：输入图像边缘的填充量，默认为 0。填充可以是字符串（如 'same' 来自动计算填充以保持输出大小不变）或整数。

6、 **dilation** ：卷积核的扩张率，用于增加卷积核的覆盖范围，默认为 1。这个目前我们也没有用到，可以暂时不用管。


#### nn.BatchNorm2d
这个就是批量的进行数据归一化处理，用于提高训练速度、稳定性和网络的泛化能力。一般就是卷积后都要跟一个nn.BatchNorm2d，这个先记住就可以。

常用的参数就是 **num_features** 输入数据的通道数。

#### nn.MaxPool2d
这个是一个二维最大池化层，举个例子来说其实就是当我们卷积池化完是一个(224,224)大小的图像的时候，我们可以利用nn.MaxPool2d池化来减小图像的大小，进而减小参数量。

这个函数的常用的参数有如下几个：

1、 **kernel_size** ：池化窗口的大小。可以是一个 **整数** ，或者 **一对整数，分别表示高度和宽度** 。

2、 **stride** ：池化窗口的步长。如果不设置，它默认与 kernel_size 相同。步长为1意味着每次滑动一个像素。

3、 **padding** ：输入特征图边缘的填充量。可以是单个数字或一对数字，分别表示垂直和水平填充。

4、 **dilation** ：池化窗口的膨胀率，用于增加感受野。

这里我们的nn.MaxPool2d(2,2)这里的两个2分别是kernel_size以及stride。

#### note
1、一般图像处理的路子都是cnn+batchnorm+relu，这都是必须。之后的pool其实是自己调节的可选。

2、我们在跑代码的时候虽然input的是[batch_size,channel,height,weight]，但是在运行nn.xx这些函数的时候其实不需要在意batch的具体大小，只要在input某个nn.xx函数的时候其input dimension对应上了即可。

In [6]:
import torch.nn.functional as F

class Network_bn(nn.Module):
    def __init__(self):
        super(Network_bn, self).__init__()
        """
        nn.Conv2d()函数：
        第一个参数（in_channels）是输入的channel数量
        第二个参数（out_channels）是输出的channel数量
        第三个参数（kernel_size）是卷积核大小
        第四个参数（stride）是步长，默认为1
        第五个参数（padding）是填充大小，默认为0
        """
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(12)
        self.pool = nn.MaxPool2d(2,2)
        self.conv4 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=0)
        self.bn4 = nn.BatchNorm2d(24)
        self.conv5 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=0)
        self.bn5 = nn.BatchNorm2d(24)
        self.fc1 = nn.Linear(24*50*50, len(classNames)) 
        #这个地方为什么是24*50*50是因为最终得到的是24个channel，每个channel图片的大小是50*50，这里之所以知道图片大小是50*50是因为self.fc1的上一层输出的图片大小就是这个。
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))      
        x = F.relu(self.bn2(self.conv2(x)))     
        x = self.pool(x)                        
        x = F.relu(self.bn4(self.conv4(x)))     
        x = F.relu(self.bn5(self.conv5(x)))  
        x = self.pool(x)  
        #print("x_shape",x.shape) torch.Size([32, 24, 50, 50])                    
        x = x.view(-1, 24*50*50)
        x = self.fc1(x)

        return x

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

model = Network_bn().to(device)
#如果我们现在是在gpu上训练，这里我们需要将model搬运到gpu上去。
model

Using cpu device


Network_bn(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(12, 12, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1))
  (bn4): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1))
  (bn5): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=60000, out_features=2, bias=True)
)

#### nn.CrossEntropyLoss()
因为我们目前处理的是一个二分类问题，因此我们的损失函数应该用到交叉熵损失函数，这里只需要注意一点就是input的数据都得是tensor数据。

#### note
这里要额外提一下，**train_loss = all_loss / num_batches** ，**train_acc = all_acc / all_size** 。

In [7]:
loss_fn    = nn.CrossEntropyLoss() # 创建损失函数
learn_rate = 1e-4 # 学习率
opt        = torch.optim.SGD(model.parameters(),lr=learn_rate)

In [8]:
# 训练循环
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)  # 训练集的大小，一共60000张图片
                                    # 显然通过dataloader.dataset可以获得所有的data,比如input图像的数据维度是[3,224,224]。
    num_batches = len(dataloader)   # 批次数目，1875（60000/32）
                                    # 显然通过dataloader可以获得batch的data，比如input图像的数据维度是[32,3,224,224]。
    train_loss, train_acc = 0, 0  # 初始化训练损失和正确率
    
    for X, y in dataloader:  # 获取图片及其标签
        X, y = X.to(device), y.to(device) #将所有的数据转移到gpu上。
        
        # 计算预测误差
        pred = model(X)          # 网络输出
        loss = loss_fn(pred, y)  # 计算网络输出和真实值之间的差距，targets为真实值，计算二者差值即为损失
        
        # 反向传播
        optimizer.zero_grad()  # grad属性归零
        loss.backward()        # 反向传播
        optimizer.step()       # 每一步自动更新
        
        # 记录acc与loss
        train_acc  += (pred.argmax(1) == y).type(torch.float).sum().item()
        #这里要表达的含义是
        #1、首先pred大小是[32,2]，因此我们要从列的角度来选出最大的哪个idx，argmax得到的是最大的数的idx，如果等于y那这个等式就为1；
        #2、之后要用.type函数将整数变成浮点数，因为在训练过程中只能使用浮点数；
        #3、(pred.argmax(1) == y).type(torch.float)这个得到的其实是只包含(0,1)的大小为(32,)的这样一个向量，之后我们求.sum()；
        #4、这时候.sum()得到的其实是tensor(xx.)，当我们用.item()的时候会将这个xx.数据取出来。
        train_loss += loss.item()
            
    train_acc  /= size
    train_loss /= num_batches

    return train_acc, train_loss

In [9]:
def test (dataloader, model, loss_fn):
    size        = len(dataloader.dataset)  # 测试集的大小，一共10000张图片
    num_batches = len(dataloader)          # 批次数目，313（10000/32=312.5，向上取整）
    test_loss, test_acc = 0, 0
    
    # 当不进行训练时，停止梯度更新，节省计算内存消耗
    with torch.no_grad():
        for imgs, target in dataloader:
            imgs, target = imgs.to(device), target.to(device)
            
            # 计算loss
            target_pred = model(imgs)
            loss        = loss_fn(target_pred, target)
            
            test_loss += loss.item()
            test_acc  += (target_pred.argmax(1) == target).type(torch.float).sum().item()

    test_acc  /= size
    test_loss /= num_batches

    return test_acc, test_loss

#### 格式化文本输出
这里是利用格式化文本输出的方式输出内容，首先这里定义了一个template模板。这使用了 Python 的字符串格式化占位符。这个模板指定了输出文本的格式，其中：

1、{:2d} 表示一个占位符，用于表示整数，宽度为2位，右对齐。

2、{:.1f} 表示一个占位符，用于表示浮点数，保留1位小数，并以百分比形式显示（因为后面有 % 符号）。

3、{:.3f} 表示一个占位符，用于表示浮点数，保留3位小数。

之后是利用template.format()将print数据显示出来。

In [11]:
epochs     = 5
train_loss = []
train_acc  = []
test_loss  = []
test_acc   = []

for epoch in range(epochs):
    model.train() #在训练的时候要用model.train()
    epoch_train_acc, epoch_train_loss = train(train_dl, model, loss_fn, opt)
    
    model.eval() #在测试的时候要用model.eval()
    epoch_test_acc, epoch_test_loss = test(test_dl, model, loss_fn)
    
    #下面这四个是记录不同epoch的信息
    train_acc.append(epoch_train_acc)
    train_loss.append(epoch_train_loss)
    test_acc.append(epoch_test_acc)
    test_loss.append(epoch_test_loss)
    
    template = ('Epoch:{:2d}, Train_acc:{:.1f}%, Train_loss:{:.3f}, Test_acc:{:.1f}%，Test_loss:{:.3f}')
    print(template.format(epoch+1, epoch_train_acc*100, epoch_train_loss, epoch_test_acc*100, epoch_test_loss))
print('Done')

Epoch: 1, Train_acc:65.2%, Train_loss:0.640, Test_acc:67.6%，Test_loss:0.602
Epoch: 2, Train_acc:71.5%, Train_loss:0.563, Test_acc:70.6%，Test_loss:0.556
Epoch: 3, Train_acc:77.6%, Train_loss:0.510, Test_acc:72.5%，Test_loss:0.523
Epoch: 4, Train_acc:77.9%, Train_loss:0.472, Test_acc:75.3%，Test_loss:0.495
Epoch: 5, Train_acc:81.1%, Train_loss:0.439, Test_acc:77.4%，Test_loss:0.486
Done


In [ ]:
import matplotlib.pyplot as plt
#隐藏警告
import warnings
warnings.filterwarnings("ignore")               #忽略警告信息
plt.rcParams['font.sans-serif']    = ['SimHei'] # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False      # 用来正常显示负号
plt.rcParams['figure.dpi']         = 100        #分辨率

epochs_range = range(epochs)
#创建一个新图形，设置图形的大小为宽 12 单位、高 3 单位。
plt.figure(figsize=(12, 3))
#将图形分割成 1 行 2 列的子图，并选择左上角（第 1 个）的子图进行绘图。
plt.subplot(1, 2, 1)
plt.plot(epochs_range, train_acc, label='Training Accuracy')
plt.plot(epochs_range, test_acc, label='Test Accuracy')
#添加图例和标题。
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

#选择右下角（第 2 个）的子图进行绘图。
plt.subplot(1, 2, 2)
plt.plot(epochs_range, train_loss, label='Training Loss')
plt.plot(epochs_range, test_loss, label='Test Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

#### torch.squeeze()&torch.unsqueeze()
squeeze()是减维度，unsqueeze()是加维度。对于一个x = torch.tensor([1, 2, 3, 4])这样(4,)的数据，当我们用a = x.unsqueeze(1)的时候得到的a的维度是(4,1)，如果是a = x.unsqueeze(0)的时候，得到的a的维度是(1,4)。squeeze()的作用是类似的，就是减维度。

除此之外还可以使用a = torch.unsqueeze(x,1) or a = torch.unsqueeze(x,0)效果和上面的一样。torch.squeeze(input_data,dim)的作用也类似，也是减维度。




In [12]:
from PIL import Image 

classes = list(total_data.class_to_idx)

def predict_one_image(image_path, model, transform, classes):
    
    test_img = Image.open(image_path).convert('RGB')
    #这里我们是利用Image函数打开一张image_path路径下的图片，并将其转换成RGB格式。
    #一般在cv领域我们处理的图像都是RGB格式的，但是一般我们input的图像数据都是jpg/png格式的，因此其本身就是以RGB格式编码的。
    #所以这里转换或者不转换都可以。
    plt.imshow(test_img)  # 展示预测的图片

    test_img = transform(test_img)#按照起初图像预处理的方式再预处理一下这张图片。
    #print(test_img.shape)
    img = test_img.to(device).unsqueeze(0)#因为我们input到model的格式要求是[batch,dim,height,width]，因此这里要增加一维度。
    #print(img.shape)
    model.eval()
    output = model(img)

    _,pred = torch.max(output,1)
    #torch.max输出两个值，number和idx。
    pred_class = classes[pred]
    print(f'预测结果是：{pred_class}')

In [16]:
# 预测训练集中的某张照片
predict_one_image(image_path='./4-data/Monkeypox/M01_01_00.jpg', 
                  model=model, 
                  transform=train_transforms, 
                  classes=classes)

预测结果是：Others


In [22]:
# 模型保存
PATH = './model.pth'  # 保存的参数文件名
torch.save(model.state_dict(), PATH)#保存model的参数

# 将参数加载到model当中，map_location表明是加载到哪个设备中
model.load_state_dict(torch.load(PATH, map_location=device))

<All keys matched successfully>